평균분산분석 1차

In [ ]:
# 메모리에 있는 모든 변수를 지우고 새롭게 시작함
all = [var for var in globals() if var[0] != "_"]
for var in all:
    del globals()[var]

In [ ]:
# 사용되는 패키지을 읽어들임 
import numpy as np
import pandas as pd
import random
import calendar
import matplotlib.pyplot as plt
from scipy.stats import mstats, skew, kurtosis
import statsmodels.api as sm
import seaborn as sns
from statsmodels.graphics.gofplots import qqline
import math
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 시장수익률과 무위험자산 수익률을 읽음
sdata = pd.read_csv("/gdrive//MyDrive/quantfinance/data.csv", index_col=0)

sdata.index = pd.to_datetime(sdata.index)
n = sdata.shape[0]
MR = sdata['RKOSPI']
RF = sdata['RF_M']

In [ ]:
# 개별자산 데이터, 전체 데이터 읽음
df = pd.read_excel("/gdrive//MyDrive/quantfinance/QuantDataR.xlsx", None, index_col=0 )
df_names = list(df.keys())

In [ ]:
#df1 = df 로딩시간을 대비하여 예비 파일 생성
df = df1

In [ ]:
# 개별주식의 총수익률 (RI) 데이터 읽음
RID = df['RI']

# Dates
date = RID.index

#firm names using 'D_Price'  이름 이외에는 다른 군더기가 없음
D_Price = df['D_P']
firm_names =  D_Price.columns

In [ ]:
############################################################################################
# Calculation of log-returns

R = [] 
print("Null stocks: ", end ='')
for i in range(0,RID.shape[1]):
    
    if RID.shape[0] - RID.iloc[:, i ].isnull().sum() < 2:   # 전체 데이터에서 NA개수를 뺀 나머지가 2 보다 작으면 계산하지 않음
        
        print (i, ", ", end='')
        R_temp = pd.Series([np.nan]*(RID.shape[0]-1))    #전체를 NA로 대체
        R.append(R_temp)
        continue
    
    lag0 = (np.log(RID.iloc[1:, i ])).reset_index(drop =True)
    lag1 = (np.log(RID.iloc[0:-1, i ])).reset_index(drop =True)
 
    R_temp =  (lag0-lag1)*100
    R.append(R_temp)
    
R = pd.concat(R,axis= 1)   # R이라는 matrix는 각 개별주식의 수익률 벡터를 순차적으로 오른편으로 덧붙여 작성. axis=0 이면 아래쪽으로 붙임
R.index = date[1:]         # 첫달 제외
R.columns = firm_names     # 회사이름
R = R.dropna(axis=1)   # 입력이 없는 열전체가 삭제, 전체기간 수익률 있는 기업만 남김

Null stocks: 25 , 747 , 

In [ ]:
# Choose ten firm for testing 10개 기업 추출
rnfirms=[0]
for i in range(1,11):
  rnfirm = random.randint(1,len(R.columns))   #Randomly Selecting a firm
  rnfirms.append(rnfirm)

print(rnfirms) #Print the selected firms
R.iloc[:,rnfirms]

[0, 378, 155, 429, 50, 192, 404, 561, 494, 47, 15]


,ABLE C&C,NI STEEL,GWANGJU SHINSEGAE,SAMSUNG CLIMATE CONTROL,CJ CGV,HANWHA,PYUNG HWA INDUSTRIAL,YUYANG D&U SUSP - SUSP.20/03/20,SK SECURITIES,CHUNIL EXPRESS,BOHAE BREWERY
Name,,,,,,,,,,,
2006-09-29,4.372000,5.798726,20.360044,14.011000,4.007311,5.509683,2.524708,-1.606686,-4.446803,13.193711,12.062739
2006-10-31,24.040686,-5.798726,1.952058,-2.678511,-12.759395,6.954992,-1.884828,2.394922,5.320016,39.803505,-0.227015
2006-11-30,-13.268377,15.201621,2.232464,12.978065,-3.239524,11.778304,-6.567791,19.926229,21.272753,17.420440,8.928542
2006-12-29,-9.720986,-6.624939,2.982445,-1.773228,-4.311486,-1.517609,-4.341328,-25.181935,-5.741156,-17.745038,-5.564945
2007-01-31,-14.786239,9.156719,-12.479537,-7.865468,-7.757860,-9.117555,6.691693,-9.121601,-11.719457,-6.361464,-6.677028
...,...,...,...,...,...,...,...,...,...,...,...
2020-05-29,-9.827651,16.846452,-3.117738,1.468682,-14.258641,-2.941186,4.176492,0.000000,-1.415535,3.296292,-4.529794
2020-06-30,0.864266,-7.649603,-3.950516,0.324547,-1.531530,9.711545,-10.171014,0.000000,31.723799,2.527355,-10.656036
2020-07-31,-10.800751,9.569048,1.092167,1.761638,-3.728175,13.691224,-3.338979,0.000000,-4.557303,3.112409,15.869332


In [ ]:
# 추정할 파라미터를 정의
SAVG = []     # 이름만 지정된 변수
SSD = []      # 표준편차
SSK = []      # 왜도
SKT = []      # 첨도

for i in range(1,12):   
    Y = (R.iloc[:,i])    # sdata의 두번째 열에서 6번쨰 열까지 다섯개의 주식, iloc는 수자로 행이나 열을 지정. 참고로 # loc는 행이나 열의 이름그대로(label)으로 지정. e.g., iloc[1] vs loc[A] 
    SAVG.append(Y.mean())
    SSD.append(Y.std())
    SSK.append(skew(Y))
    SKT.append(kurtosis(Y))

    print(R.iloc[:,i].name, ":   Average Excess Return:", Y.mean(),",   STD:", Y.std(), ",   Skewness:", skew(Y), ",   Kurtosis:", kurtosis(Y))     # 결과물을 프린트함
    

AK HOLDINGS :   Average Excess Return: 0.4347633038745198 ,   STD: 14.861654091992577 ,   Skewness: -0.18407126709456484 ,   Kurtosis: 1.25016879925502
AMOREPACIFIC :   Average Excess Return: 0.8555203348532854 ,   STD: 10.027938508458417 ,   Skewness: -0.5570419026924047 ,   Kurtosis: 4.23273551001735
AMOREPACIFIC GROUP :   Average Excess Return: 0.9598390699280802 ,   STD: 11.580700528820218 ,   Skewness: -0.3775900568655674 ,   Kurtosis: 2.544574861344217
ANAM ELECTRONICS :   Average Excess Return: 1.0972069977570342 ,   STD: 18.326807455669467 ,   Skewness: 2.1250804218552704 ,   Kurtosis: 13.528618337667798
APROGEN KIC :   Average Excess Return: -0.8848314558912455 ,   STD: 21.710453726550135 ,   Skewness: 1.8008924628985956 ,   Kurtosis: 17.984497636159922
APROGEN PHARMACEUTICALS :   Average Excess Return: -1.3546648409085857 ,   STD: 21.666221059020074 ,   Skewness: 1.1591427671402454 ,   Kurtosis: 5.094648886915683
ASIA HOLDINGS :   Average Excess Return: 0.3820433270815406 ,  

In [ ]:
SAVG = pd.Series(SAVG)   # series나 DataFrame 형식의 데이터만 concatenate가능
SSD = pd.Series(SSD)
SSK = pd.Series(SSK)
SKT = pd.Series(SKT)

Inv_Cov = np.linalg.inv(R.iloc[:,1:12].cov())   # 공분산의 역행렬 (시그마역행렬)
InWgts = Inv_Cov@SAVG                              # 
InWgts = pd.Series(InWgts/ InWgts.sum())

R1 = pd.concat([SAVG, SSD, SSK, SKT, InWgts], axis=1)   # df 라는 matrix는 SAVG, SSD, SSK, SKT, InWgts 의 벡터를 순차적으로 오른편으로 덧붙여 작성. axis=0 이면 아래쪽으로 붙임
R1.columns = ['SAVG', 'SSD', 'SSK', 'SKT', 'lnWgts']    # df matrix 의 열 이름 부여
R1.index = R.columns[1:12]                           # df matrix의 행 이름은 각 주식의 회사 이름으로 부여

print("주식별 초과평균수익률, STD, 왜도, 첨도, 그리고 투자비중")     # 결과물을 프린트함
R1


주식별 초과평균수익률, STD, 왜도, 첨도, 그리고 투자비중


,SAVG,SSD,SSK,SKT,lnWgts
AK HOLDINGS,0.434763,14.861654,-0.184071,1.250169,0.278055
AMOREPACIFIC,0.855520,10.027939,-0.557042,4.232736,1.516477
AMOREPACIFIC GROUP,0.959839,11.580701,-0.377590,2.544575,0.322380
ANAM ELECTRONICS,1.097207,18.326807,2.125080,13.528618,0.679085
APROGEN KIC,-0.884831,21.710454,1.800892,17.984498,-0.490269
APROGEN PHARMACEUTICALS,-1.354665,21.666221,1.159143,5.094649,-0.331426
ASIA HOLDINGS,0.382043,9.528323,-0.344595,2.283970,0.575666
ASIA PAPER MNFG.,0.900890,12.275957,0.101480,2.516817,0.650871
ASIANA AIRLINES,-0.235146,11.912959,1.345295,4.473052,-0.435338
AUK,-0.020770,12.896748,0.499161,1.836671,-0.270729


In [ ]:
corr_R1 = R1.corr(method='pearson')     # 상관계수 pearson이 default, spearman은 서열변수에서 쓰긔
print("The correlation Matrix is:")
print(corr_R1, "\n")
# 기대수익률 극대화 최적 포트폴리오

The correlation Matrix is:
            SAVG       SSD       SSK       SKT    lnWgts
SAVG    1.000000 -0.575695 -0.219745 -0.018268  0.877755
SSD    -0.575695  1.000000  0.707272  0.629979 -0.501236
SSK    -0.219745  0.707272  1.000000  0.789375 -0.305581
SKT    -0.018268  0.629979  0.789375  1.000000  0.002883
lnWgts  0.877755 -0.501236 -0.305581  0.002883  1.000000 

